In [25]:
import pandas as pd
import numpy as np
from collections import namedtuple

In [2]:
df = pd.read_csv('../data/one_travel_chain.csv')
def build_chain(group):
    chains = []
    current_chain = [group['grid_id_o'].iloc[0]]
    
    for i in range(len(group) - 1):
        current_chain.append(group['grid_id_d'].iloc[i])
        # Check if the next 'o' is different from the current 'd'
        if group['grid_id_d'].iloc[i] != group['grid_id_o'].iloc[i + 1]:
            chains.append(current_chain)
            current_chain = [group['grid_id_o'].iloc[i + 1]]
    
    # Append the last destination and the final chain
    current_chain.append(group['grid_id_d'].iloc[-1])
    chains.append(current_chain)
    
    return chains

In [11]:
# Group by 'date' and apply the chain-building function
grouped1 = df[df['date']<df['migrt']]
grouped2 = df[df['date']>df['migrt']]
grouped1 = grouped1.groupby('date').apply(build_chain).reset_index(name='travel_chain')
grouped2 = grouped2.groupby('date').apply(build_chain).reset_index(name='travel_chain')

In [20]:
# Define the namedtuple type
TravelData = namedtuple('TravelChain', ['date', 'travel_chain'])

# Convert each row to a namedtuple
namedtuples_list1 = [TravelData(row.date, row.travel_chain[0]) for _, row in grouped1.iterrows()]
namedtuples_list2 = [TravelData(row.date, row.travel_chain[0]) for _, row in grouped2.iterrows()]

In [34]:
np.save('../data/before_migrt.npy',np.array(namedtuples_list1))

np.save('../data/after_migrt.npy',np.array(namedtuples_list2))

C:\Users\301\AppData\Local\Temp\ipykernel_18052\3119323862.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  np.save('../data/before_migrt.npy',np.array(namedtuples_list1))
C:\Users\301\AppData\Local\Temp\ipykernel_18052\3119323862.py:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  np.save('../data/after_migrt.npy',np.array(namedtuples_list2))


In [42]:
import json
dicts_list1 = [nt._asdict() for nt in namedtuples_list1]
with open('../data/before_migrt.json', 'w') as file:
    json.dump(dicts_list1, file, indent=4)

dicts_list2 = [nt._asdict() for nt in namedtuples_list2]
with open('../data/after_migrt.json', 'w') as file:
    json.dump(dicts_list2, file, indent=4)

In [43]:
with open('../data/before_migrt.json', 'r') as file:
    loaded_dicts_list1 = json.load(file)
loaded_namedtuples_list1 = [TravelData(**d) for d in loaded_dicts_list1]
loaded_namedtuples_list1

[TravelChain(date=20190101, travel_chain=[20850.0, 17637.0, 22994.0]),
 TravelChain(date=20190102, travel_chain=[20850.0, 11854.0, 20850.0]),
 TravelChain(date=20190103, travel_chain=[15794.0, 11854.0, 16557.0, 22994.0]),
 TravelChain(date=20190104, travel_chain=[22994.0, 11854.0, 15483.0, 20850.0]),
 TravelChain(date=20190106, travel_chain=[22994.0, 15103.0, 16201.0]),
 TravelChain(date=20190107, travel_chain=[20850.0, 11854.0, 12966.0, 11854.0, 10472.0]),
 TravelChain(date=20190108, travel_chain=[18359.0, 11854.0, 18359.0, 20850.0]),
 TravelChain(date=20190109, travel_chain=[22994.0, 11854.0, 16492.0, 11854.0, 14719.0, 18359.0]),
 TravelChain(date=20190110, travel_chain=[20850.0, 11854.0, 20850.0]),
 TravelChain(date=20190111, travel_chain=[20850.0, 11854.0, 33669.0, 22994.0]),
 TravelChain(date=20190112, travel_chain=[22994.0, 11854.0, 9708.0, 11854.0, 35100.0, 20850.0]),
 TravelChain(date=20190113, travel_chain=[23351.0, 20850.0, 11854.0, 22994.0]),
 TravelChain(date=20190114, trav